Raster CLI and API sidebyside

In [1]:
import os
os.chdir(r"D:\1. PROJECT\Mangroves CDO\analysis - 9-30-2025\buffer")
os.getcwd()

from osgeo import gdal, ogr
import numpy as np
import  pandas as pd

In [2]:
from osgeo import gdal, ogr
import numpy as np
import  pandas as pd

#input files
raster_path = 'non_ahp_co_reproj.tif'
shapefile_path = 'bufer_250m.shp'

ds = gdal.Open(raster_path)
band = ds.GetRasterBand(1)
gt = ds.GetGeoTransform()
nodata = band.GetNoDataValue()

# Pixel Area
pixel_width = gt[1]
pixel_height = abs(gt[5])
pixel_area = pixel_width * pixel_height

#Opn shp
driver = ogr.GetDriverByName("ESRI Shapefile")
shp = driver.Open(shapefile_path)
layer = shp.GetLayer()

# Mask
mask_ds = gdal.GetDriverByName("MEM").Create(
    '', ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Byte)
mask_ds.SetGeoTransform(gt)
mask_ds.SetProjection(ds.GetProjection())

# Rasterize buffer
gdal.RasterizeLayer(mask_ds, [1], layer, burn_values=[1])

# Read data
mask = mask_ds.GetRasterBand(1).ReadAsArray()
data = band.ReadAsArray()

# Apply mask
masked = np.where(mask == 1, data, np.nan)

# Get unique classes and pixel counts
unique, counts = np.unique(masked[~np.isnan(masked)], return_counts=True)

# Compute area
areas_m2 = counts * pixel_area
areas_ha = areas_m2 / 10000

# Put into table
df = pd.DataFrame({
    "class": unique.astype(int),
    "area_m2": areas_m2,
    "area_ha": areas_ha
})

print(df)
df.to_csv("non_ahp.csv", index=False)

c:\Users\deleo\miniconda3\envs\webgisbackend\Lib\site-packages\osgeo\gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


                 class       area_m2     area_ha
0 -9223372036854775808  1.254094e+05   12.540941
1                   11  4.181822e+06  418.182208
2                   12  3.365239e+06  336.523874
3                   13  1.782688e+06  178.268832
4                   21  5.578779e+04    5.578779
5                   22  9.355284e+05   93.552835
6                   23  1.341622e+06  134.162213
7                   31  5.817962e+02    0.058180
8                   32  2.592225e+04    2.592225
9                   33  8.067575e+04    8.067575


C:\Users\deleo\AppData\Local\Temp\ipykernel_1840\1280776801.py:49: RuntimeWarning: invalid value encountered in cast
  "class": unique.astype(int),


In [ ]:
!ogrinfo C:\Users\deleo\Downloads\buffer\bufer_250m.shp -al -so

### Clipping Raster in the buffer


In [29]:
#files
raster_path = 'ahp_co_reproj.tif'
shapefile_path = 'bufer_250m.shp'
output_path = "ahp_250m.tif"

try: 
  shp = ogr.Open(shapefile_path)
  layer= shp.GetLayer()
  print("Clipping with", layer.GetName())
  
  gdal.Warp(
    output_path,
    raster_path,
    cutlineDSName=shapefile_path,
    cropToCutline="true"
  )
  print("Clipping successfully to " + output_path)
  
except Exception as e:
  print(f"An unexpected error occurred: {e}")


Clipping with bufer_250m
Clipping successfully to ahp_250m.tif


Calculating the Area

In [17]:
import geopandas as gpd

path = "ahp_union.shp"
output_csv = "ahp_union_report.csv"

try:
  gdf = gpd.read_file(path)
  gdf =gdf.to_crs(epsg=32651)
  gdf['area_m2'] = gdf.geometry.area
  print(gdf.columns)
  agg = gdf.groupby(['ADM4_EN', 'level_v2'])['area_m2'].sum().reset_index()
  print(agg)
  agg.to_csv(output_csv, index=False)
  print("CSV save successfully")
  
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Index(['DN', 'score', 'ADM4_EN', 'level_v2', 'Area', 'geometry', 'area_m2'], dtype='object')
      ADM4_EN level_v2        area_m2
0      Agusan      Low  168925.695099
1      Agusan   Medium  211253.259792
2     Bayabas     High   45780.881302
3     Bayabas      Low  215327.023242
4     Bayabas   Medium  302921.178118
5      Bonbon     High   56628.680694
6      Bonbon      Low  188708.818042
7      Bonbon   Medium  241654.493557
8        Bugo     High    3955.478157
9        Bugo      Low  145757.161013
10       Bugo   Medium  148660.293568
11      Bulua     High    1460.151258
12      Bulua      Low   96020.083928
13      Bulua   Medium  177281.027943
14     Cugman     High   35700.742118
15     Cugman      Low  137891.405805
16     Cugman   Medium  211246.885118
17       Gusa     High   10559.816059
18       Gusa      Low  182231.314096
19       Gusa   Medium  181678.185930
20  Kauswagan     High  323838.978385
21  Kauswagan      Low   31526.593712
22  Kauswagan   Medium  418404.62